In [ ]:
%matplotlib qt
import pyxdf
import mne
import numpy as np
from mne.datasets import misc
import os
import matplotlib.pyplot as plt
import matplotlib
import time
print(os.getcwd())

fnames = ["sub-LeviShield_ses-S002_task-DefaultASSR_acq-assrV3Layout1Ultracortex_run-001_eeg.xdf"]
raws = []
for fnameIndex in range(len(fnames)): 
    streams, header = pyxdf.load_xdf(fnames[fnameIndex])
    # get sampling rate
    sfreq = float(streams[1]["info"]["nominal_srate"][0])

    # get data
    data = streams[1]["time_series"].T
    dataTimestamps = streams[1]["time_stamps"].T
    # get markers
    markers = streams[0]["time_series"].T
    markerTimestamps = streams[0]["time_stamps"].T

    #in case of there is a switch between stim channel and data channels
    if  data.shape[0] != 8:
        data = streams[0]["time_series"].T
        dataTimestamps = streams[0]["time_stamps"].T
        sfreq = float(streams[0]["info"]["nominal_srate"][0])
        markers = streams[1]["time_series"].T
        markerTimestamps = streams[1]["time_stamps"].T
        assert data.shape[0] == 8,  fnames[fnameIndex] + " has different channel amounts" # check that we have what we think we have (8 eeg channels)

    

    # get together the event channel with the data, make a "stim 014" type channel as annotation: time, duration, event type 
    # add markers: What needs to be done: we have markers as several events, and their timestamp. then, we have the data
    # and their timestamps, to the raw object we should add one more channel, initially full of zeros with the length of the measurement
    
    
    # This is because microvolts are being measured, but mne plot expects values in volts.
    markers = markers[0]
    data *= 1e-6 # uV -> V and preamp gain. IDK what will preamp gain do. (multiply / divide by gain? 24?)

    markerArr = np.zeros_like(data[0])
    
    for markerid in range(len(markers)):
        absolute_differences = np.abs(dataTimestamps - markerTimestamps[markerid])
        closest_index = np.argmin(absolute_differences)
        markerArr[closest_index] = markers[markerid]

    # TEST: this gives the amount of events, and also their index. divided by the sampling freq, you should see the time differences of
    #each part of the experiment. 
    nonzero_indices = np.nonzero(markerArr)
    num_nonzero = len(nonzero_indices[0])
    print("uniques", np.unique(markerArr))
    print("Number of nonzero elements:", num_nonzero)
    print("Indices of nonzero elements:", nonzero_indices[0] / sfreq)

    #add marker arr to data
    combined = np.c_[data.T,markerArr].T

    # create info object set channel names
    info = mne.create_info(["T6", "T4", "F8", "C4", "C3", "T5", "T3", "F7", "stim"], sfreq, ["eeg", "eeg","eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "stim"])
    raw = mne.io.RawArray(combined, info)
    raw.set_montage("standard_1020")

    #preliminary data analyzis: (fancy term for plotting it and looking at it). Task: review data and select a consistently bad channel for exclusion
    print("Task: review data and select a consistently bad channel for exclusion.")
    
    raw.plot(title=fnames[fnameIndex] + ", Task: review data and select consistently bad channels", duration=1)
    raw.compute_psd().plot()
    plt.title(fnames[fnameIndex])
    plt.show()
    raws.append((raw, fnames[fnameIndex]))

In [ ]:
epochs = []
for rawpack in raws: 
    raw = rawpack[0]
    fname = rawpack[1]
    low_cut = 0.1
    high_cut = 50
    filtered_data = raw.copy().filter(low_cut,high_cut,method='iir')
    notch_freqs = [50, 100]
    filtered_data = filtered_data.notch_filter(freqs=notch_freqs, filter_length='auto', method='fir', notch_widths=6)

    #get events    
    events = mne.find_events(filtered_data, stim_channel="stim")
    #events: 1: start rest, 2: start assr, 3: artifact
    #make epochs
    stimEpochs = mne.Epochs(filtered_data, events, event_id=2, tmin=-2, tmax=5, baseline=(-2, 0), preload=True)
    epochs.append((stimEpochs, fnames[fnameIndex]))

    stimEpochs.plot(title=fname+"filtered")
    stimEpochs.compute_psd().plot()
    plt.title(fname)
    plt.show()

In [ ]:
# in this cell i want to epoch this up, and then remove some based on some conditions (significantly more avg peak to peak than normal, spikes, artifact events)
# reject bad epochs?